In [74]:
import tifffile
import os
import numpy as np
import stackview

In [2]:
DATA_PATH = "/mnt/efs/shared_data/restorators/spheres"
source_list = os.listdir(os.path.join(DATA_PATH, 'images'))
idx = 0
source_path = os.path.join(DATA_PATH, 'images', source_list[idx])
source = tifffile.imread(source_path)

In [12]:
sorted(source_list)

['0000_sphere.tiff',
 '0001_sphere.tiff',
 '0002_sphere.tiff',
 '0003_sphere.tiff',
 '0004_sphere.tiff',
 '0005_sphere.tiff',
 '0006_sphere.tiff',
 '0007_sphere.tiff',
 '0008_sphere.tiff',
 '0009_sphere.tiff',
 '0010_sphere.tiff',
 '0011_sphere.tiff',
 '0012_sphere.tiff',
 '0013_sphere.tiff',
 '0014_sphere.tiff',
 '0015_sphere.tiff',
 '0016_sphere.tiff',
 '0017_sphere.tiff',
 '0018_sphere.tiff',
 '0019_sphere.tiff',
 '0020_sphere.tiff',
 '0021_sphere.tiff',
 '0022_sphere.tiff',
 '0023_sphere.tiff',
 '0024_sphere.tiff',
 '0025_sphere.tiff',
 '0026_sphere.tiff',
 '0027_sphere.tiff',
 '0028_sphere.tiff',
 '0029_sphere.tiff',
 '0030_sphere.tiff',
 '0031_sphere.tiff',
 '0032_sphere.tiff',
 '0033_sphere.tiff',
 '0034_sphere.tiff',
 '0035_sphere.tiff',
 '0036_sphere.tiff',
 '0037_sphere.tiff',
 '0038_sphere.tiff',
 '0039_sphere.tiff',
 '0040_sphere.tiff',
 '0041_sphere.tiff',
 '0042_sphere.tiff',
 '0043_sphere.tiff',
 '0044_sphere.tiff',
 '0045_sphere.tiff',
 '0046_sphere.tiff',
 '0047_sphere

In [109]:
class ImageTransformer:
    def __init__(self, src_dir, dest_dir):
        self.src_dir = src_dir
        self.src_im_dir = os.path.join(src_dir, 'images')
        self.src_ob_dir = os.path.join(src_dir, 'objects')
        self.dest_dir = dest_dir
        self.dest_im_dir = os.path.join(dest_dir, 'images')
        self.dest_ob_dir = os.path.join(dest_dir, 'objects')

        # Ensure destination directory exists or create it
        if not os.path.exists(self.dest_im_dir):
            os.makedirs(self.dest_im_dir)
        if not os.path.exists(self.dest_ob_dir):
            os.makedirs(self.dest_ob_dir)

    def crop_object(self, crop_box):
        """
        Crop images in the last two dimensions based on the provided crop_box and save to dest_ob_dir.

        Parameters:
        - crop_box: tuple of four integers (left, upper, right, lower) defining the box to crop.
        """
        # List all TIFF files in the source directory of objects
        files = sorted(os.listdir(self.src_ob_dir))
        for file in files:
            # Construct full file paths
            src_path = os.path.join(self.src_ob_dir, file)
            dest_path = os.path.join(self.dest_ob_dir, file)

            # Read the TIFF file
            image = tifffile.imread(src_path)

            # Validate image shape
            if image.shape != (4, 8, 32, 32):
                print(f"Skipping {file} due to unexpected shape {image.shape}.")
                continue

            # Crop images in the last two dimensions
            left, upper, right, lower = crop_box
            cropped_image = image[:, :, upper:lower, left:right]

            # Save the cropped TIFF file to the destination directory
            tifffile.imwrite(dest_path, cropped_image)

        print(f"Successfully transformed and saved TIFF images from {self.src_ob_dir} to {self.dest_ob_dir}.")

    def resave_image(self):
        """
        Resave the light field images in a new directory.
        """
        # List all TIFF files in the source directory of images
        files = sorted(os.listdir(self.src_im_dir))
        for file in files:
            # Construct full file paths
            src_path = os.path.join(self.src_im_dir, file)
            dest_path = os.path.join(self.dest_im_dir, file)

            # Read the TIFF file
            image = tifffile.imread(src_path)

            # Save the TIFF file to the destination directory
            tifffile.imwrite(dest_path, image)

        print(f"Successfully transformed and saved TIFF images from {self.src_im_dir} to {self.dest_im_dir}.")

    def verify_crop_volume_empty(self, crop_box):
        """
        Crop images in the last two dimensions based on the provided crop_box.
        Then, check that the cropped region only contains zero birefringence values.

        Parameters:
        - crop_box: tuple of four integers (left, upper, right, lower) defining the box to crop.
        """
        num_nonempty = 0
        files = sorted(os.listdir(self.src_ob_dir))
        for file in files:
            # Construct full file paths
            src_path = os.path.join(self.src_ob_dir, file)

            # Read the TIFF file
            image = tifffile.imread(src_path)

            # Crop images in the last two dimensions
            left, upper, right, lower = crop_box
            cropped_image = image[:, :, upper:lower, left:right]
            
            # Check for nonzero birefringence values
            delta_n = cropped_image[0, ...]
            nonzero_bool = np.any(delta_n != 0)

            if nonzero_bool == True:
                count = np.count_nonzero(delta_n)
                num_nonempty += 1
                print(f"Volume {file} has {count} nonzero birefringence values.")
                # break

        print(f"Successfully checked that the cropped region {crop_box} from the objects in" +
              f"{self.src_ob_dir} have only zero birefringence values, except for {num_nonempty} objects.")

    def verify_volume_empty_after_zero_fill(self, crop_box):
        """
        Crop images in the last two dimensions based on the provided crop_box.
        Then, replace that region of the original volume with zero birefringence.
        Lastly, verify that the volume only contains zero birefringence values.

        Parameters:
        - crop_box: tuple of four integers (left, upper, right, lower)
                    defining the region to replace with zeros.
        """
        num_nonempty = 0
        files = sorted(os.listdir(self.src_ob_dir))
        for file in files:
            # Construct full file paths
            src_path = os.path.join(self.src_ob_dir, file)

            # Read the TIFF file
            image = tifffile.imread(src_path)

            delta_n_fill_zero = image[0, ...].copy()
            left, upper, right, lower = crop_box
            delta_n_fill_zero[:, upper:lower, left:right] = 0
            cropped_image = image[:, :, upper:lower, left:right]
            
            # Check for nonzero birefringence values
            nonzero_bool = np.any(delta_n_fill_zero != 0)

            if nonzero_bool == True:
                count = np.count_nonzero(delta_n_fill_zero)
                num_nonempty += 1
                print(f"Volume {file} has {count} nonzero birefringence values.")
                # break

        print(f"Successfully checked that the cropped region {crop_box} from the objects in" +
              f"{self.src_ob_dir} have only zero birefringence values, except for {num_nonempty} objects.")

    def verify_zdepth_empty(self):
        """
        Check if the top of bottom z depth layer has nonzero values.

        Parameters:
        - crop_box: tuple of four integers (left, upper, right, lower)
                    defining the region to replace with zeros.
        """
        num_nonempty = 0
        files = sorted(os.listdir(self.src_ob_dir))
        for file in files:
            # Construct full file paths
            src_path = os.path.join(self.src_ob_dir, file)

            # Read the TIFF file
            image = tifffile.imread(src_path)

            delta_n_top = image[0, 0, :, :]
            delta_n_bottom = image[0, -1, :, :]
            
            # Check for nonzero birefringence values
            nonzero_bool_top = np.any(delta_n_top != 0)

            if nonzero_bool_top == True:
                count = np.count_nonzero(delta_n_top)
                num_nonempty += 1
                print(f"Volume {file} top layer has {count} nonzero birefringence values.")
                # break

            # Check for nonzero birefringence values
            nonzero_bool_bottom = np.any(delta_n_bottom != 0)

            if nonzero_bool_bottom == True:
                count = np.count_nonzero(delta_n_bottom)
                num_nonempty += 1
                print(f"Volume {file} bottom layer has {count} nonzero birefringence values.")
                # break

        print(f"Successfully checked that the objects in" +
              f"{self.src_ob_dir} have only zero birefringence values, " +
              f"except for {num_nonempty} top or bottom layers of objects.")

In [110]:
transformer = ImageTransformer("/mnt/efs/shared_data/restorators/spheres", "/mnt/efs/shared_data/restorators/spheres_cropped_tmp_0_13")
first_region = (0, 0, 11, 12)
second_region = (19, 20, 32, 32)
transformer.verify_crop_volume_empty(first_region)
transformer.verify_crop_volume_empty(second_region)
transformer.verify_zdepth_empty()

Successfully checked that the cropped region (0, 0, 11, 12) from the objects in/mnt/efs/shared_data/restorators/spheres/objects have only zero birefringence values, except for 0 objects.
Successfully checked that the cropped region (19, 20, 32, 32) from the objects in/mnt/efs/shared_data/restorators/spheres/objects have only zero birefringence values, except for 0 objects.
Volume 0000_sphere.tiff top layer has 5 nonzero birefringence values.
Volume 0002_sphere.tiff top layer has 5 nonzero birefringence values.
Volume 0010_sphere.tiff bottom layer has 5 nonzero birefringence values.
Volume 0012_sphere.tiff bottom layer has 9 nonzero birefringence values.
Volume 0020_sphere.tiff top layer has 5 nonzero birefringence values.
Volume 0024_sphere.tiff bottom layer has 9 nonzero birefringence values.
Volume 0026_sphere.tiff bottom layer has 5 nonzero birefringence values.
Volume 0028_sphere.tiff bottom layer has 9 nonzero birefringence values.
Volume 0041_sphere.tiff top layer has 9 nonzero b

In [54]:
transformer = ImageTransformer("/mnt/efs/shared_data/restorators/spheres", "/mnt/efs/shared_data/restorators/spheres_cropped_cube")
object_region = (11, 12, 19, 20)
transformer.crop_object(object_region)
transformer.resave_image()

Successfully transformed and saved TIFF images from /mnt/efs/shared_data/restorators/spheres/objects to /mnt/efs/shared_data/restorators/spheres_cropped_cube/objects.
Successfully transformed and saved TIFF images from /mnt/efs/shared_data/restorators/spheres/images to /mnt/efs/shared_data/restorators/spheres_cropped_cube/images.


In [111]:
transformer = ImageTransformer("/mnt/efs/shared_data/restorators/spheres", "/mnt/efs/shared_data/restorators/spheres_11by11")
object_region = (10, 10, 21, 21)
transformer.verify_volume_empty_after_zero_fill(object_region)
transformer.crop_object(object_region)
transformer.resave_image()

Successfully checked that the cropped region (10, 10, 21, 21) from the objects in/mnt/efs/shared_data/restorators/spheres/objects have only zero birefringence values, except for 0 objects.
Successfully transformed and saved TIFF images from /mnt/efs/shared_data/restorators/spheres/objects to /mnt/efs/shared_data/restorators/spheres_10by10/objects.
Successfully transformed and saved TIFF images from /mnt/efs/shared_data/restorators/spheres/images to /mnt/efs/shared_data/restorators/spheres_10by10/images.


In [115]:
os.rename("/mnt/efs/shared_data/restorators/spheres_10by10", "/mnt/efs/shared_data/restorators/spheres_11by11")

In [116]:
target_list = sorted(os.listdir(os.path.join("/mnt/efs/shared_data/restorators/spheres_11by11", 'objects')))

In [117]:
target_list

['0000_sphere.tiff',
 '0001_sphere.tiff',
 '0002_sphere.tiff',
 '0003_sphere.tiff',
 '0004_sphere.tiff',
 '0005_sphere.tiff',
 '0006_sphere.tiff',
 '0007_sphere.tiff',
 '0008_sphere.tiff',
 '0009_sphere.tiff',
 '0010_sphere.tiff',
 '0011_sphere.tiff',
 '0012_sphere.tiff',
 '0013_sphere.tiff',
 '0014_sphere.tiff',
 '0015_sphere.tiff',
 '0016_sphere.tiff',
 '0017_sphere.tiff',
 '0018_sphere.tiff',
 '0019_sphere.tiff',
 '0020_sphere.tiff',
 '0021_sphere.tiff',
 '0022_sphere.tiff',
 '0023_sphere.tiff',
 '0024_sphere.tiff',
 '0025_sphere.tiff',
 '0026_sphere.tiff',
 '0027_sphere.tiff',
 '0028_sphere.tiff',
 '0029_sphere.tiff',
 '0030_sphere.tiff',
 '0031_sphere.tiff',
 '0032_sphere.tiff',
 '0033_sphere.tiff',
 '0034_sphere.tiff',
 '0035_sphere.tiff',
 '0036_sphere.tiff',
 '0037_sphere.tiff',
 '0038_sphere.tiff',
 '0039_sphere.tiff',
 '0040_sphere.tiff',
 '0041_sphere.tiff',
 '0042_sphere.tiff',
 '0043_sphere.tiff',
 '0044_sphere.tiff',
 '0045_sphere.tiff',
 '0046_sphere.tiff',
 '0047_sphere

In [105]:
OG_DATA_PATH = "/mnt/efs/shared_data/restorators/spheres"
CUBE_DATA_PATH = "/mnt/efs/shared_data/restorators/spheres_cropped_cube"
target_list = sorted(os.listdir(os.path.join(OG_DATA_PATH, 'objects')))
idx = 4
target_path = os.path.join(DATA_PATH, 'objects', target_list[idx])
target = tifffile.imread(target_path)

target_cube_list = sorted(os.listdir(os.path.join(CUBE_DATA_PATH, 'objects')))
idx = 0
target_cube_path = os.path.join(CUBE_DATA_PATH, 'objects', target_cube_list[idx])
target_cube = tifffile.imread(target_cube_path)



In [72]:
target.shape
target_cube.shape
crop_box = object_region
left, upper, right, lower = crop_box
cropped_image = target[:, :, upper:lower, left:right]

In [75]:
cropped_image.shape
target_list[idx]
delta_n = target[0, ...]
stackview.orthogonal(delta_n, continuous_update=True, zoom_factor=10)

In [76]:
stackview.orthogonal(cropped_image[0, ...], continuous_update=True, zoom_factor=10)

In [113]:
my_crop = target[:, :, 10:21, 10:21]
stackview.orthogonal(my_crop[0, ...], continuous_update=True, zoom_factor=10)

In [114]:
my_crop.shape

(4, 8, 11, 11)

In [ ]:
crop_box = ()

In [83]:
delta_n_fill_zero = target[0, ...].copy()
left, upper, right, lower = (11, 12, 19, 20)
delta_n_fill_zero[:, upper:lower, left:right] = 0
# delta_n_fill_zero[:, 9:12, 9:12] = 0
# delta_n_fill_zero[:, 9:12, 9:12] = 0
np.any(delta_n_fill_zero != 0)

True

In [6]:
transformer = ImageTransformer("/mnt/efs/shared_data/restorators/spheres", "/mnt/efs/shared_data/restorators/spheres_cropped_tmp_0_13")
region = (0, 0, 13, 13)
# region = (19, 19, 32, 32)
cropped = transformer.crop_volume(region)
ret = cropped[0, ...]
np.any(ret != 0)

NameError: name 'ImageTransformer' is not defined

In [47]:
cropped = transformer.crop_volume((10, 10, 24, 24))

False

In [36]:
np.any(ret != 0)

False